## Modelling

In [52]:
#Importing Required Libraries
import numpy as np
import pandas as pd
import dask.dataframe as dk
import calendar
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler,Normalizer
from scipy.sparse import csr_matrix,hstack
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
from lightgbm import LGBMRegressor
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import pickle
import optuna

In [53]:
dtype={'id'       :     'object', 
    'item_id'     :  'int64', 
    'dept_id'     :  'int8', 
    'cat_id'      :  'int8', 
    'store_id'    :  'int8', 
    'state_id'    :  'int8', 
    'd'           :  'object', 
    'sales'       :  'int16',  
    'date'        : 'object', 
   'wday'        :  'int8',  
   'month'       :  'int8',  
   'year'        :  'int16',  
   'event_name_1' : 'int8', 
   'event_type_1' : 'int8', 
   'event_name_2' : 'int8', 
   'event_type_2' : 'int8', 
    'snap':'int8',
  'sell_price'   : 'float16',
       'price_change':'float16',
   'week_number'  : 'int8',  
   'season'       : 'object', 
   'quater_start' : 'int8',  
   'quater_end'   : 'int8',  
   'month_start'  : 'int8',  
   'month_end'    : 'int8',  
   'year_start'   : 'int8',  
   'year_end'     : 'int8',  
   'group'        : 'int8',  
   'no_events'    : 'object', 
   'holiday'      : 'object',
    'week_number':'int8',
       'season':'int8',
       'quater_start':'int8',
       'quater_end':'int8',
       'month_start':'int8',
       'month_end':'int8',
       'year_start':'int8',
       'year_end':'int8',
       'roll_7_shift_28_mean':'float16',
       'roll_14_shift_28_mean':'float16',
       'roll_30_shift_28_mean':'float16',
       'roll_60_shift_28_mean':'float16',
       'roll_360_shift_28_mean':'float16',
       'roll_7_shift_28_std':'float16',
       'roll_14_shift_28_std':'float16',
       'roll_30_shift_28_std':'float16',
       'roll_60_shift_28_std':'float16',
       'roll_360_shift_28_std':'float16',
       'direct_ewm':'float16',
       'direct_lag_28':'int16',
       'direct_lag_35':'int16',
       'direct_lag_42':'int16',
       'direct_lag_49':'int16',
       'direct_lag_56':'int16',
       'direct_lag_63':'int16',
       'direct_lag_70':'int16',
       'direct_lag_77':'int16',
       'direct_lag_84':'int16',
       'direct_lag_91':'int16',
       'direct_lag_98':'int16',
       'min_price':'float16',
       'max_price':'float16',
       'mean_price':'float16',
       'std_price':'float16',
       'price_norm_1':'float16',
       'price_norm_2':'float16',
       'price_norm_3':'float16',
      }

In [54]:
train=pd.read_feather('Intermediate Data/train1.feather')
cv=pd.read_feather('Intermediate Data/cv1.feather')
test=pd.read_feather('Intermediate Data/test1.feather')
final_test=pd.read_feather('Intermediate Data/final_test1.feather')

In [55]:
for col, typ in dtype.items():
    if col in train.columns:
        train[col] = train[col].astype(typ)
        cv[col] = cv[col].astype(typ)
        test[col] = test[col].astype(typ)
        final_test[col] = final_test[col].astype(typ)

In [56]:
# WRMSSE Calculation
def caluclate_WRMSSE(actual,predicted,train,weights,h,n):
    '''This function is used to calculate RMSSE'''
    num=((actual-predicted)**2).sum(axis=1)/h
    denom=(train[:,1:]-train[:,:-1])**2
    denom=denom.sum(axis=1)/(n-1)
    return (num/denom)**0.5

In [57]:
def get_model_performances(model,train,cv,test,X_cv,X_test):
    '''This Function is used to get WRMSSE that is used in this Case Study as a Metric For CV and Test Data'''
    #For CV Data
    cv['prices']=cv['sales']*cv['sell_price']
    total_sales=cv.groupby('id').sum()
    total=sum(total_sales['sales'])
    weight={}
    for i in total_sales.index:
        weight[i]=total_sales.loc[i]['sales']/total
    train1=pd.concat([train,cv])
    train1.sort_values(['id','date'],inplace=True)
    train1.fillna(0,inplace=True)
    df=train1.pivot_table(index=['id','state_id','store_id','cat_id','dept_id','item_id'],columns='d',values='sales')
    df.reset_index(level=[0,1,2,3,4,5],inplace=True)
    df.fillna(0,inplace=True)
    del train1
    import gc
    X_cv = cv[m_lgb.feature_name()]
    cv['pred_sales']=model.predict(X_cv)
    df1=cv.pivot_table(index=['id'],columns='d',values='pred_sales')
    dic={}
    for j,i in enumerate(range(1886,1914)):
        dic['d_'+str(i)]='F'+str(j+1)
    df1=df1.rename(columns=dic) 
    df1.reset_index(level=[0],inplace=True)
    dd=df.merge(df1,on='id')
    dd['weight']=dd['id'].apply(lambda x:weight[x])
    l=['d_'+str(i) for i in range(1,1914)]
    l1=['F'+str(i) for i in range(1,29)]
    agg_level={2:['state_id'],3:['store_id'],4:['cat_id'],5:['dept_id'],6:['state_id','cat_id'],\
           7:['state_id','dept_id'],8:['store_id','cat_id'],9:['store_id','dept_id'],10:['item_id'],11:['item_id','state_id']}
    agg=pd.DataFrame(dd[l+l1].sum()).transpose()
    agg['weight']=1/12
    agg['level']=1
    col=agg.columns
    for level in agg_level:
        temp_df=dd.groupby(by=agg_level[level]).sum().reset_index(drop=True)
        temp_df['weight']/=12
        temp_df['level']=level
        agg = pd.concat([agg, temp_df[col]], ignore_index=True)
    dd['weight']/=12
    dd['level']=12    
    agg = pd.concat([agg, dd[col]], ignore_index=True)
    actual=agg[['d_1886', 'd_1887', 'd_1888', 'd_1889', 'd_1890', 'd_1891', 'd_1892',
       'd_1893', 'd_1894', 'd_1895', 'd_1896', 'd_1897', 'd_1898', 'd_1899',
       'd_1900', 'd_1901', 'd_1902', 'd_1903', 'd_1904', 'd_1905', 'd_1906',
       'd_1907', 'd_1908', 'd_1909', 'd_1910', 'd_1911', 'd_1912', 'd_1913']].values
    predicted=agg[['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21',
       'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']].values
    training=agg[['d_'+str(i) for i in range(1,1886)]].values
    weights=agg['weight'].values
    rmsse=caluclate_WRMSSE(actual,predicted,training,weights,28,1886)
    cv_wrsmme=np.sum(rmsse*weights)
    cv.drop(['pred_sales'],axis=1,inplace=True)
    del actual,predicted,training,weights,agg
    
    #For Test data
    test['prices']=test['sales']*test['sell_price']
    total_sales=test.groupby('id').sum()
    total=sum(total_sales['sales'])
    weight={}
    for i in total_sales.index:
        weight[i.replace('evaluation','validation')]=total_sales.loc[i]['sales']/total
    test['id']=test['id'].apply(lambda x:x.replace('evaluation','validation'))
    train1=pd.concat([train,cv,test])
    train1.sort_values(['id','date'],inplace=True)
    train1.fillna(0,inplace=True)
    df=train1.pivot_table(index=['id','state_id','store_id','cat_id','dept_id','item_id'],columns='d',values='sales')
    df.reset_index(level=[0,1,2,3,4,5],inplace=True)
    df.fillna(0,inplace=True)
    del train1
    import gc
    test['pred_sales']=model.predict(X_test)
    df1=test.pivot_table(index=['id'],columns='d',values='pred_sales')
    dic={}
    for j,i in enumerate(range(1914,1942)):
        dic['d_'+str(i)]='F'+str(j+1)
    df1=df1.rename(columns=dic) 
    df1.reset_index(level=[0],inplace=True)
    dd=df.merge(df1,on='id')
    dd['weight']=dd['id'].apply(lambda x:weight[x])
    l=['d_'+str(i) for i in range(1,1942)]
    l1=['F'+str(i) for i in range(1,29)]
    agg_level={2:['state_id'],3:['store_id'],4:['cat_id'],5:['dept_id'],6:['state_id','cat_id'],\
           7:['state_id','dept_id'],8:['store_id','cat_id'],9:['store_id','dept_id'],10:['item_id'],11:['item_id','state_id']}
    agg=pd.DataFrame(dd[l+l1].sum()).transpose()
    agg['weight']=1/12
    agg['level']=1
    col=agg.columns
    for level in agg_level:
        temp_df=dd.groupby(by=agg_level[level]).sum().reset_index(drop=True)
        temp_df['weight']/=12
        temp_df['level']=level
        agg = pd.concat([agg, temp_df[col]], ignore_index=True)
    dd['weight']/=12
    dd['level']=12    
    agg = pd.concat([agg, dd[col]], ignore_index=True)
    actual=agg[['d_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919',
       'd_1920', 'd_1921', 'd_1922', 'd_1923', 'd_1924', 'd_1925',
       'd_1926', 'd_1927', 'd_1928', 'd_1929', 'd_1930', 'd_1931',
       'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937',
       'd_1938', 'd_1939', 'd_1940', 'd_1941']].values
    predicted=agg[['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21',
       'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']].values
    training=agg[['d_'+str(i) for i in range(1,1914)]].values
    weights=agg['weight'].values
    rmsse=caluclate_WRMSSE(actual,predicted,training,weights,28,1914)
    test_wrsmme=np.sum(rmsse*weights)

    print("CV WRMSSE=",cv_wrsmme)
    print("Test WRMSSE=",test_wrsmme)

In [58]:
def get_model_performances_for_store_wise_trained_model(train,cv,test,cv_pred,test_pred):
    '''This Function is used to get WRMSSE that is used in this Case Study as a Metric For CV and Test Data where model is trained according to store id(Mainly used for Catabosst)'''
    #For CV Data
    cv['prices']=cv['sales']*cv['sell_price']
    total_sales=cv.groupby('id').sum()
    total=sum(total_sales['sales'])
    weight={}
    for i in total_sales.index:
        weight[i]=total_sales.loc[i]['sales']/total
    train1=pd.concat([train,cv])
    train1.sort_values(['id','date'],inplace=True)
    train1.fillna(0,inplace=True)
    df=train1.pivot_table(index=['id','state_id','store_id','cat_id','dept_id','item_id'],columns='d',values='sales')
    df.reset_index(level=[0,1,2,3,4,5],inplace=True)
    df.fillna(0,inplace=True)
    del train1
    import gc
    
    df1=cv_pred
    dd=df.merge(df1,on='id')
    dd['weight']=dd['id'].apply(lambda x:weight[x])
    l=['d_'+str(i) for i in range(1,1914)]
    l1=['F'+str(i) for i in range(1,29)]
    agg_level={2:['state_id'],3:['store_id'],4:['cat_id'],5:['dept_id'],6:['state_id','cat_id'],\
           7:['state_id','dept_id'],8:['store_id','cat_id'],9:['store_id','dept_id'],10:['item_id'],11:['item_id','state_id']}
    agg=pd.DataFrame(dd[l+l1].sum()).transpose()
    agg['weight']=1/12
    agg['level']=1
    col=agg.columns
    for level in agg_level:
        temp_df=dd.groupby(by=agg_level[level]).sum().reset_index(drop=True)
        temp_df['weight']/=12
        temp_df['level']=level
        agg = pd.concat([agg, temp_df[col]], ignore_index=True)
    dd['weight']/=12
    dd['level']=12    
    agg = pd.concat([agg, dd[col]], ignore_index=True)
    actual=agg[['d_1886', 'd_1887', 'd_1888', 'd_1889', 'd_1890', 'd_1891', 'd_1892',
       'd_1893', 'd_1894', 'd_1895', 'd_1896', 'd_1897', 'd_1898', 'd_1899',
       'd_1900', 'd_1901', 'd_1902', 'd_1903', 'd_1904', 'd_1905', 'd_1906',
       'd_1907', 'd_1908', 'd_1909', 'd_1910', 'd_1911', 'd_1912', 'd_1913']].values
    predicted=agg[['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21',
       'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']].values
    training=agg[['d_'+str(i) for i in range(1,1886)]].values
    weights=agg['weight'].values
    rmsse=caluclate_WRMSSE(actual,predicted,training,weights,28,1886)
    cv_wrsmme=np.sum(rmsse*weights)
    del actual,predicted,training,weights,agg
    
    #For Test data
    test['prices']=test['sales']*test['sell_price']
    total_sales=test.groupby('id').sum()
    total=sum(total_sales['sales'])
    weight={}
    for i in total_sales.index:
        weight[i.replace('evaluation','validation')]=total_sales.loc[i]['sales']/total
    test['id']=test['id'].apply(lambda x:x.replace('evaluation','validation'))
    train1=pd.concat([train,cv,test])
    train1.sort_values(['id','date'],inplace=True)
    train1.fillna(0,inplace=True)
    df=train1.pivot_table(index=['id','state_id','store_id','cat_id','dept_id','item_id'],columns='d',values='sales')
    df.reset_index(level=[0,1,2,3,4,5],inplace=True)
    df.fillna(0,inplace=True)
    del train1
    import gc
    df1=test_pred
    df1['id']=df1['id'].apply(lambda x:x.replace('evaluation','validation'))
    dd=df.merge(df1,on='id')
    dd['weight']=dd['id'].apply(lambda x:weight[x])
    l=['d_'+str(i) for i in range(1,1942)]
    l1=['F'+str(i) for i in range(1,29)]
    agg_level={2:['state_id'],3:['store_id'],4:['cat_id'],5:['dept_id'],6:['state_id','cat_id'],\
           7:['state_id','dept_id'],8:['store_id','cat_id'],9:['store_id','dept_id'],10:['item_id'],11:['item_id','state_id']}
    agg=pd.DataFrame(dd[l+l1].sum()).transpose()
    agg['weight']=1/12
    agg['level']=1
    col=agg.columns
    for level in agg_level:
        temp_df=dd.groupby(by=agg_level[level]).sum().reset_index(drop=True)
        temp_df['weight']/=12
        temp_df['level']=level
        agg = pd.concat([agg, temp_df[col]], ignore_index=True)
    dd['weight']/=12
    dd['level']=12    
    agg = pd.concat([agg, dd[col]], ignore_index=True)
    actual=agg[['d_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919',
       'd_1920', 'd_1921', 'd_1922', 'd_1923', 'd_1924', 'd_1925',
       'd_1926', 'd_1927', 'd_1928', 'd_1929', 'd_1930', 'd_1931',
       'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937',
       'd_1938', 'd_1939', 'd_1940', 'd_1941']].values
    predicted=agg[['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21',
       'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']].values
    training=agg[['d_'+str(i) for i in range(1,1914)]].values
    weights=agg['weight'].values
    rmsse=caluclate_WRMSSE(actual,predicted,training,weights,28,1914)
    test_wrsmme=np.sum(rmsse*weights)

    print("CV WRMSSE=",cv_wrsmme)
    print("Test WRMSSE=",test_wrsmme)

### LGBM Model

In [59]:
#categorical Data's
columns=['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id','wday', 'month', 'year',
         'event_name_1','event_type_1', 'event_name_2', 'event_type_2','snap','week_number',
         'quater_start', 'quater_end', 'month_start','month_end', 'year_start',
         'year_end','season']

In [60]:
data=lgb.Dataset(train.drop(['id','sales','date','d'],axis=1),label=train['sales'],categorical_feature=columns,free_raw_data=False)

In [61]:
data_cv=lgb.Dataset(cv.drop(['id','sales','date','d'],axis=1),label=cv['sales'],categorical_feature=columns,free_raw_data=False )

In [70]:
def objective(trial):
    params = {
        "objective": "tweedie",
        "metric": "rmse",
        "tweedie_variance_power": trial.suggest_float("tweedie_variance_power", 1.05, 1.3),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.05),
        "num_leaves": trial.suggest_int("num_leaves", 128, 512),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 256, 2047),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 0.95),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 0.95),
        "bagging_freq": 1,
        "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 1.0),
        "boost_from_average": True,
        "max_bin": 127,
        "bin_construct_sample_cnt": 20000000,
        "force_row_wise": True,
        "verbosity": 1,
        "nthread": 8,
        "seed": 42
    }

    # Sample training subset for speed (2M rows)
    sample_train = train.iloc[:2_000_000]
    X_train = sample_train.drop(['id', 'sales', 'date', 'd'], axis=1)
    y_train = sample_train['sales']
    
    X_valid = cv.drop(['id', 'sales', 'date', 'd'], axis=1)
    y_valid = cv['sales']

    train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=columns, free_raw_data=False)
    valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=columns, free_raw_data=False)

    model = lgb.train(
        params,
        train_data,
        valid_sets=[train_data, valid_data],
        valid_names=['train', 'valid'],
        num_boost_round=3000,
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(period=100)
        ]
    )

    preds = model.predict(X_valid, num_iteration=model.best_iteration)
    rmse = mean_squared_error(y_valid, preds, squared=False)
    return rmse

In [71]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

[I 2025-05-02 20:44:52,920] A new study created in memory with name: no-name-f380042f-b37b-4750-afd5-bbfff9655b50


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Total Bins 4377
[LightGBM] [Info] Number of data points in the train set: 2000000, number of used features: 39
[LightGBM] [Info] Start training from score 0.580102
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 2.90497	valid's rmse: 2.73165
Early stopping, best iteration is:
[57]	train's rmse: 3.18316	valid's rmse: 2.70152


[I 2025-05-02 20:45:03,781] Trial 0 finished with value: 2.7015166741092274 and parameters: {'tweedie_variance_power': 1.2905498889152502, 'learning_rate': 0.04787241406623165, 'num_leaves': 161, 'min_data_in_leaf': 990, 'feature_fraction': 0.7158920962306441, 'bagging_fraction': 0.9394593381415659, 'lambda_l2': 0.19066062854224186}. Best is trial 0 with value: 2.7015166741092274.


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Total Bins 4377
[LightGBM] [Info] Number of data points in the train set: 2000000, number of used features: 39
[LightGBM] [Info] Start training from score 0.580102
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 3.28945	valid's rmse: 2.72557
Early stopping, best iteration is:
[131]	train's rmse: 3.14749	valid's rmse: 2.70466


[I 2025-05-02 20:45:27,584] Trial 1 finished with value: 2.7046574303384543 and parameters: {'tweedie_variance_power': 1.2846491908512974, 'learning_rate': 0.024334161002437883, 'num_leaves': 336, 'min_data_in_leaf': 1869, 'feature_fraction': 0.7980324720168311, 'bagging_fraction': 0.7578760944875959, 'lambda_l2': 0.8863543365354135}. Best is trial 0 with value: 2.7015166741092274.


...

[I 2025-05-02 20:54:59,088] Trial 27 finished with value: 2.668554183750474 and parameters: {'tweedie_variance_power': 1.2682664257984304, 'learning_rate': 0.04371580535250708, 'num_leaves': 288, 'min_data_in_leaf': 1802, 'feature_fraction': 0.9156733350245699, 'bagging_fraction': 0.6308936002180039, 'lambda_l2': 0.22530003867926113}. Best is trial 23 with value: 2.6558062009486374.


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Total Bins 4377
[LightGBM] [Info] Number of data points in the train set: 2000000, number of used features: 39
[LightGBM] [Info] Start training from score 0.580102
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 2.99049	valid's rmse: 2.70991
Early stopping, best iteration is:
[88]	train's rmse: 3.05256	valid's rmse: 2.70376


[I 2025-05-02 20:55:16,832] Trial 28 finished with value: 2.7037631837119425 and parameters: {'tweedie_variance_power': 1.2081551937707495, 'learning_rate': 0.033723925578004375, 'num_leaves': 175, 'min_data_in_leaf': 929, 'feature_fraction': 0.8731711552617695, 'bagging_fraction': 0.7262594810430304, 'lambda_l2': 0.45283213439127645}. Best is trial 23 with value: 2.6558062009486374.


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Total Bins 4377
[LightGBM] [Info] Number of data points in the train set: 2000000, number of used features: 39
[LightGBM] [Info] Start training from score 0.580102
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 2.90606	valid's rmse: 2.74122
Early stopping, best iteration is:
[69]	train's rmse: 3.08706	valid's rmse: 2.70602


[I 2025-05-02 20:55:34,449] Trial 29 finished with value: 2.70602307955554 and parameters: {'tweedie_variance_power': 1.291133077659551, 'learning_rate': 0.04601260436505606, 'num_leaves': 210, 'min_data_in_leaf': 1505, 'feature_fraction': 0.8407937160040513, 'bagging_fraction': 0.9408037633033202, 'lambda_l2': 0.554790626776351}. Best is trial 23 with value: 2.6558062009486374.


In [72]:
best_params = study.best_params
best_params.update({
    "objective": "tweedie",
    "metric": "rmse",
    "boost_from_average": True,
    "max_bin": 127,
    "bin_construct_sample_cnt": 20000000,
    "force_row_wise": True,
    "verbosity": 1,
    "nthread": 8,
    "seed": 42
})

In [73]:
m_lgb = lgb.train(
    best_params,
    data,
    valid_sets=[data, data_cv],
    valid_names=['train', 'valid'],
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=100)  # same as verbose_eval=1
    ]
)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Total Bins 6049
[LightGBM] [Info] Number of data points in the train set: 45174237, number of used features: 44
[LightGBM] [Info] Start training from score 0.356321
Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 2.68306	valid's rmse: 2.1763
Did not meet early stopping. Best iteration is:
[100]	train's rmse: 2.68306	valid's rmse: 2.1763


In [81]:
m_lgb.save_model('lgbm_optuna_model.txt')

In [75]:
get_model_performances(m_lgb,train,cv,test,cv.drop(['id','sales','date','d'],axis=1),test.drop(['id','sales','date','d'],axis=1))

CV WRMSSE= 0.6402907900201394
Test WRMSSE= 0.7770103022417408


### Submission

In [76]:
test['pred_sales']=m_lgb.predict(test.drop(['id','sales','date','d','prices', 'pred_sales'],axis=1))
final_test['pred_sales']=m_lgb.predict(final_test.drop(['id','sales','date','d',],axis=1))

In [77]:
df=test.pivot_table(index='id',columns='date',values='pred_sales')
df.reset_index(level=0,inplace=True)
df['id']=df['id'].apply(lambda x:x.replace('evaluation','validation'))
dic={}
l=test['date'].unique()
for i,day in enumerate(l):
    dic[day]='F'+str(i+1)
df.rename(columns=dic,inplace=True)

In [78]:
df1=final_test.pivot_table(index='id',columns='d',values='pred_sales')
df1.reset_index(level=0,inplace=True)
dic={}
for i,day in enumerate(range(1942,1970)):
    dic['d_'+str(day)]='F'+str(i+1)
df1.rename(columns=dic,inplace=True) 

In [79]:
df2=pd.concat([df1,df])

In [80]:
df2.to_csv('lgbm_submission_2.csv',index=False)